## Agreement between readers

Following: 
Ranganathan, P., Pramesh, C. S., & Aggarwal, R. (2017). Common pitfalls in statistical analysis: Measures of agreement. Perspectives in clinical research, 8(4), 187–191. https://doi.org/10.4103/picr.PICR_123_17
https://doi.org/10.4103%2Fpicr.PICR_123_17

For metric variables ->  ICC

For ordinal -> Cohen's Kappa weighted version 

For categorical -> Cohen's Kappa not weighted version 



In [1]:
import os
import sys
import pandas as pd
pd.options.mode.copy_on_write = True 

from pathlib import Path
import cdata_utils
import numpy as np
import cdata_utils.utils
import datetime


import json

#import cdata_utils.preprocess.read_and_clean_tabular
from cdata_utils.project_specific.psvd import (
    read_and_clean_PSVD_data__BL_consensus,
    categorize_PSVD_data,
    exclude_patients,
    reorder_some_categorical_values, 
    table1_psvd, 
    table1_psvd_spleen, 
    descriptive_df_from_masks, 
    masks_for_endpoint_1__decompensation, 
    masks_for_endpoint_2__death,
    make_y_delta,
    drop_non_numeric_columns,
    table_of_valid_entries, 
    univariate_cox_ph_summary, 
    normalize_df,
    columns_important_variables_BL_FU,
    categorize_PSVD_data_, 
    flatten, 
    read_and_clean_PSVD_data__BL_FU_consensus,
)

from cdata_utils.descriptive.basic_stats import (
    describe, 
    convert_bool_bool_to_int,
)

import cdata_utils.preprocess
import cdata_utils.project_specific
import cdata_utils.project_specific.psvd


import lifelines
from lifelines import CoxPHFitter
from lifelines.datasets import load_rossi

from sklearn.preprocessing import StandardScaler

# path info: 
if "cwatzenboeck" in os.getcwd(): # desktop 
    data_path = Path("/home/cwatzenboeck/Dropbox/work/data/livermodel/PSVD/")
    data_path_output=Path("/home/cwatzenboeck/data/psvd/output_coxph/")
else: # laptop 
    data_path = Path("/home/clemens/Dropbox/work/data/livermodel/PSVD/")
    # data_path = Path("/home/clemens/projects/project_liver_model/data/PSVD")
    

    
from sklearn.metrics import cohen_kappa_score


import pingouin as pg

In [ ]:
# CODE

from cdata_utils.project_specific.psvd import (
    rename_columns_by_prefix,
    read_renaming_dict
)


def read_and_clean_PSVD_data__BL_FU_consensus(data_path: Path | str, 
                                           verbose=False, 
                                           file_name = "data_PSVD_unified_1.xlsx" #"data_PSVD_orig.xlsx"
                                           ):
    data_origin_path = data_path / file_name
    dfo = pd.read_excel(data_origin_path)
    df = dfo.copy()
    df = df.drop(columns=["Name", "Prename", "DOB"])

    # rename inconsistent column names: 
    df = rename_columns_by_prefix(df, prefix_old="BL_1 ", prefix_new="BL1 ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="BL_2 ", prefix_new="BL2 ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="BL_1", prefix_new="BL1", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="BL 1_", prefix_new="BL1 ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="BL_2", prefix_new="BL2", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="BL_", prefix_new="BL ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="BL-", prefix_new="BL ", verbose=verbose)

    df = rename_columns_by_prefix(df, prefix_old="FU_1 ", prefix_new="FU1 ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="FU_2 ", prefix_new="FU2 ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="FU_1", prefix_new="FU1", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="FU 1_", prefix_new="FU1 ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="FU_2", prefix_new="FU2", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="FU_", prefix_new="FU ", verbose=verbose)
    df = rename_columns_by_prefix(df, prefix_old="FU-", prefix_new="FU ", verbose=verbose)

    return df





def get_BL1_BL2_columns(df, regex_filters, chill=True):
    rows = []
    for f in regex_filters:
        c1 = list(df.filter(regex="^BL1.*"+f).columns)
        c2 = list(df.filter(regex="^BL2.*"+f).columns)
        if not chill:
            assert len(c1) == 1
            assert len(c2) == 1
            c1 = c1[0]
            c2 = c2[0]
        rows.append({"lc1": len(c1), "lc2": len(c2), "c1": c1,  "c2": c2})
    return pd.DataFrame(rows)




# Note that one could use the weighted version for ordinal data
def cohen_kappa_df(df, c1: str, c2: str):
    df_ = df[[c1, c2]].dropna()
    y1 = df_[c1]
    y2 = df_[c2]
    kappa = cohen_kappa_score(y1, y2)
    return pd.DataFrame([{"column 1": c1, "column 2": c2, "cohen_kappa": kappa}])


def icc_two_columns(df, c1, c2, id_column = "ID"):
    df_1 = df[[id_column, c1]].rename(columns = {c1: "Scores"})
    df_2 = df[[id_column, c2]].rename(columns = {c2: "Scores"})
    df_1["Rater"] = 1
    df_2["Rater"] = 2

    df_ = pd.concat([df_1, df_2]).reset_index().drop(columns=["index"])
    icc = pg.intraclass_corr(data=df_, targets=id_column, raters='Rater',
                            ratings='Scores', nan_policy = "omit")
    return icc


def extract_icc_column_as_dict(icc_, mask_column="Type", icc_type = "ICC2"):
    
    _, dct = next(icc_[icc_[mask_column] == icc_type].reset_index().iterrows())
    pre = icc_type
    dct = {f"{pre}:  {k}": dct[k] for k in dct.keys() if k != "index"}
    return dct

def icc_stats_df(df_pairs, icc_type = "ICC2", calculate_icc=False, calculate_kappa=False, calculate_kappa_liniar = False, save_sets=False): 
    rows= []
    for i, (_, row) in enumerate(df_pairs.iterrows()):
        c1 = row["c1"]
        c2 = row["c2"]
        
        dct = {"c1": c1, "c2": c2}
        print(f"Comparing: '{c1}' vs '{c2}'")
        if save_sets:
            dct = dct | {"set c1": set(df[c1]), 
                         "set c2": set(df[c2])}
        
        
        df_ = df[[c1, c2]].dropna()
        y1 = df_[c1]
        y2 = df_[c2]
        
        if calculate_kappa:
            kappa = cohen_kappa_score(y1, y2)
            dct = dct | {"cohen kappa (not weighted)": kappa}
            
        if calculate_kappa_liniar:
            kappa_linear  = cohen_kappa_score(y1, y2, weights="linear")
            dct = dct | {"cohen kappa (linear weighted)": kappa_linear}
            
        if calculate_icc:
            icc_ = icc_two_columns(df, c1, c2)
            dct_icc = extract_icc_column_as_dict(icc_, icc_type=icc_type)
            dct = dct | dct_icc
            dct = dct | {"ICC all": icc_}
            
        rows.append(dct)  
        
    df_icc_non_metric2 = pd.DataFrame(rows)
    return df_icc_non_metric2


In [ ]:

df = read_and_clean_PSVD_data__BL_FU_consensus(data_path=data_path, file_name = "data_PSVD_unified_3.xlsx")





In [ ]:

regex_filters = [
    "Spleen",
    "Ascites",
    "_SPSS",
    "L-SPSS",
    "intrahep./LPV/RPV",
    "extrahep.",
    "SMV/SV",
    "PV overall extent",
    #
    "abnormalities", #"Intrahepatic portal vein abnormalities",
    ## "Liver morphology",
    "segment 1",
    "segment IV",
    "Atrophy/hypertrophy complex",
    "FNH-like lesions",
    "shunts",
    "TPMT", 
    "Splanchnic thrombosis",
    "Intrahepatic portal abnormalities",
    "Location"
]

# categories which work
regex_filters_ordinal_categorical = [
    "Ascites",
    "_SPSS",
    "L-SPSS",
    "intrahep./LPV/RPV",
    # "extrahep.",
    "SMV/SV",
    ## "Liver morphology",
    "segment 1",
    "segment IV",
    "Atrophy/hypertrophy complex",
    "shunts",
        "extrahep.",
        "PV overall extent",  # not sure why this crashes
]

# categories which need further preprocessing
regex_filters_ordinal_categorical_2 = [
    "FNH-like lesions", 
    "Splanchnic thrombosis",  # needs further preprocessing
    "Intrahepatic portal abnormalities",  # needs further preprocessing
    "Location"  # needs further processing
]


regex_filters_metric = [
    "Spleen",
    "TPMT"
]




In [ ]:


# get dataframe column names with BL1, BL2 for non metric variables
df_pairs = get_BL1_BL2_columns(df, regex_filters_ordinal_categorical, chill=True)
assert (df_pairs["lc1"] == 1).all(), "lc1 should be 1"
assert (df_pairs["lc2"] == 1).all(), "lc2 should be 1"
df_pairs = df_pairs.drop(columns=["lc1", "lc2"])
df_pairs["c1"] = df_pairs["c1"].apply(lambda x: x[0])
df_pairs["c2"] = df_pairs["c2"].apply(lambda x: x[0])
# df_extra = pd.DataFrame([
#     {"c1": "BL1_Liver morphology (0=normal, 1=abnormal)",
#      "c2": "BL2 Liver morphology"  
#      }])
# df_pairs = pd.concat([df_pairs, df_extra]).reset_index().drop(columns=["index"])
df_pairs_non_metric = df_pairs.copy()


# get dataframe column names with BL1, BL2 for non metric variables
df_pairs = get_BL1_BL2_columns(df, regex_filters_ordinal_categorical_2, chill=True)
assert (df_pairs["lc1"] == 1).all(), "lc1 should be 1"
assert (df_pairs["lc2"] == 1).all(), "lc2 should be 1"
df_pairs = df_pairs.drop(columns=["lc1", "lc2"])
df_pairs["c1"] = df_pairs["c1"].apply(lambda x: x[0])
df_pairs["c2"] = df_pairs["c2"].apply(lambda x: x[0])
df_pairs_non_metric_2 = df_pairs.copy()


# get dataframe column names with BL1, BL2 for metric variables
df_pairs = get_BL1_BL2_columns(df, regex_filters_metric, chill=True)
assert (df_pairs["lc1"] == 1).all(), "lc1 should be 1"
assert (df_pairs["lc2"] == 1).all(), "lc2 should be 1"
df_pairs = df_pairs.drop(columns=["lc1", "lc2"])
df_pairs["c1"] = df_pairs["c1"].apply(lambda x: x[0])
df_pairs["c2"] = df_pairs["c2"].apply(lambda x: x[0])
df_pairs_metric = df_pairs.copy()



In [ ]:
def calculate_agreement_metric_variables():
    ICC_results_metric = []
    icc_type = "ICC2"

    index = 0; 
    c1, c2  = df_pairs_metric.loc[index,:].to_numpy()
    print("Comparing: ", c1, "with ", c2)
    # exclude spleenectomy: 
    m = df[c1] == "splenectomy"
    m_ = df[c2] == "splenectomy"
    assert(m == m_).all()
    icc_ = icc_two_columns(df[~m].astype({c1: 'float', c2: 'float'}), c1, c2)
    dct_icc = extract_icc_column_as_dict(icc_, icc_type=icc_type)

    ICC_results_metric.append({"c1": c1,
                            "c2": c2,
                            "ICC all": icc_} | dct_icc)



    index = 1; 
    c1, c2  = df_pairs_metric.loc[index,:].to_numpy()
    print("Comparing: ", c1, "with ", c2)
    icc_ = icc_two_columns(df, c1, c2)
    dct_icc = extract_icc_column_as_dict(icc_, icc_type=icc_type)

    ICC_results_metric.append({"c1": c1,
                            "c2": c2,
                            "ICC all": icc_} | dct_icc)


    ICC_results_metric = pd.DataFrame(ICC_results_metric)
    return ICC_results_metric


agreement_results_metric = calculate_agreement_metric_variables()
agreement_results_metric

In [ ]:
# non metric variables which work out of the box
df_tmp = pd.DataFrame([{"c1": 'BL1_Liver morphology (0=normal, 1=abnormal)', 
               "c2": 'BL2_Liver morphology'}])
df_tmp = pd.concat([df_pairs_non_metric, df_tmp])

agreement_results_non_metric = icc_stats_df(df_tmp, calculate_kappa=True, calculate_kappa_liniar=True)
agreement_results_non_metric

In [ ]:
# All variables which need some extra preprocessing: 
display(df_pairs_non_metric_2)

In [ ]:
i=0
c1, c2 = df_pairs_non_metric_2.iloc[i, :].to_numpy()
print("c1", c1, "\nc2", c2)
print("Unique values: ", set(list(df[c1]) + list(df[c2])))

y = df[c1]
y1 = np.select([y==0, y==1, y=="only pv phase"], [0,1,2], default=np.nan)
y = df[c2]
y2 = np.select([y==0, y==1, y=="only pv phase"], [0,1,2], default=np.nan)
kappa = cohen_kappa_score(y1, y2)

r = {"c1": c1, 
     "c2": c2, 
     "cohen kappa (not weighted)": kappa, 
     "notes": "The entry 'only pv phase' was treated as a seperate category. "
     }

agreement_results_non_metric_other = pd.DataFrame([r])
agreement_results_non_metric_other


In [ ]:
i=1
c1, c2 = df_pairs_non_metric_2.iloc[i, :].to_numpy()
print("c1", c1, "\nc2", c2)
print("Unique values: ", set(list(df[c1]) + list(df[c2])))

y1 = df[c1]
y2 = df[c2]

# check that non nulls are in the data
assert (y1.isnull().any(), y2.isnull().any()) == (False, False) 

i = 1
y1_1 = y1.apply(lambda x: str(i) in str(x))
y2_1 = y2.apply(lambda x: str(i) in str(x))

i = 2
y1_2 = y1.apply(lambda x: str(i) in str(x))
y2_2 = y2.apply(lambda x: str(i) in str(x))

rows = []
rows.append({"c1": c1, "c2": c2, 
            "cohen kappa (not weighted)": cohen_kappa_score(y1_1, y2_1), 
            "notes": "Compare '1' vs 'not 1'" 
            })

rows.append({"c1": c1, "c2": c2, 
            "cohen kappa (not weighted)": cohen_kappa_score(y1_2, y2_2), 
            "notes": "Compare '2' vs 'not 2'" 
            })


agreement_results_non_metric_other_ = pd.DataFrame(rows)
agreement_results_non_metric_other = pd.concat([agreement_results_non_metric_other, agreement_results_non_metric_other_]).reset_index().drop(columns=["index"])


In [ ]:
i=2
c1, c2 = df_pairs_non_metric_2.iloc[i, :].to_numpy()
print("c1", c1, "\nc2", c2)
print("Unique values: ", set(list(df[c1]) + list(df[c2])))

y1 = df[c1]
y2 = df[c2]

# check that non nulls are in the data
assert (y1.isnull().any(), y2.isnull().any()) == (False, False) 

rows = []
for i in range(1, 5):
    y1_ = y1.apply(lambda x: str(i) in str(x))
    y2_ = y2.apply(lambda x: str(i) in str(x))
    rows.append({"c1": c1, "c2": c2, 
                "cohen kappa (not weighted)": cohen_kappa_score(y1_, y2_), 
                "notes": f"Compare '{i}' vs 'not {i}'" 
                })


agreement_results_non_metric_other_ = pd.DataFrame(rows)
agreement_results_non_metric_other = pd.concat([agreement_results_non_metric_other, agreement_results_non_metric_other_]).reset_index().drop(columns=["index"])

agreement_results_non_metric_other

In [ ]:
i=3
c1, c2 = df_pairs_non_metric_2.iloc[i, :].to_numpy()
print("c1", c1, "\nc2", c2)
print("Unique values: ", set(list(df[c1]) + list(df[c2])))

y1 = df[c1]
y2 = df[c2]
assert (y1.isnull().any(), y2.isnull().any()) == (False, False) 

y1_ = y1!=0
y2_ = y2!=0
{"c1": c1, "c2": c2, 
                "cohen kappa (not weighted)": cohen_kappa_score(y1_, y2_), 
                "notes": f"Compare as binary category (any type) vs. none;   any type (PV, SV, ...) -> 1; 0 -> 0" 
                }


agreement_results_non_metric_other_ = pd.DataFrame(rows)
agreement_results_non_metric_other = pd.concat([agreement_results_non_metric_other, agreement_results_non_metric_other_]).reset_index().drop(columns=["index"])

agreement_results_non_metric_other

In [ ]:
# save data
df_results_all = pd.concat([agreement_results_metric, agreement_results_non_metric, agreement_results_non_metric_other]).reset_index().drop(columns=["index"])
df_results_all[['c1', 'c2', 'notes', 'cohen kappa (not weighted)', 'cohen kappa (linear weighted)', 'ICC2:  ICC', 
                'ICC2:  Description', 'ICC2:  F', 'ICC2:  df1', 'ICC2:  df2', 'ICC2:  pval', 'ICC2:  CI95%']].to_excel(data_path_output / "reader_comparison_statistics.xlsx", index=False)